# Imports

In [1]:
%load_ext autoreload
%autoreload 2
import sys
from nlppen.extraccion.utils.Txt2Numbers import Txt2Numbers
from nlppen.analisis import Analisis
from nlppen.seleccion import Seleccion
from nlppen.spark_udfs import solo_portanto, spark_get_spacy
from nlppen.sentencias_estructurales import SentenciasEstructurales
from pyspark.sql import SparkSession
from pyspark.sql.functions import length

import pandas as pd

## Initialize spark context

In [2]:
spark = (SparkSession
         .builder
         .appName("Transforming sentences")
         .config("spark.num.executors", "1")
         .config("spark.executor.memory", "12g")
         .config("spark.executor.cores", "2")
         .config("spark.driver.memory", "10g")
         .config("spark.memory.offHeap.enabled", True)
         .config("spark.memory.offHeap.size", "64g")
         .config("spark.sql.execution.arrow.pyspark.enabled", "true")
         .config("spark.sql.legacy.timeParserPolicy", "LEGACY")
         .config("spark.sql.legacy.parquet.int96RebaseModeInWrite", "CORRECTED")
         .getOrCreate())


sc = spark.sparkContext
sc.addFile('/home/jovyan/Work/ej/paquetes/nlppen/nlppen.zip')
sc.uiWebUrl

21/11/24 16:02:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


'http://69abb479a5e1:4040'

# Buscar terminos en la sección de por lo tanto de la sentencia y aplicar filtro del dataset

In [5]:

anno_inf = 2010
anno_sup = 2022

terminos = {
    'seguimiento': [r'\bseguimiento\b'],
    'se ordena': [r'\bse ordena\b', r'\bse le ordena\b', r'\bse les ordena\b'],
    'se condena': [r'\bse condena\b', r'\bse le condena\b', r'\bse les condena\b'],
    'plan': [r'\bplan\b'],
    'plazo': [r'\bplazo\b']
}


seleccion = Seleccion(terminos, spark, parquet_path='../../src/datasets/complete', datasets_path='./datasets/estructurales2', filtro=f'anno >= {anno_inf} AND anno < {anno_sup}')
seleccion.filtrar_sentencias(preprocess=solo_portanto, keepRowEmpty=True, partition_cols=['anno'], precargar=False)

estructurales = SentenciasEstructurales(seleccion)

seleccion.sdf.count()

151269

In [10]:
spark.read.parquet('../../src/datasets/complete').filter('termino_ext == "Con lugar" OR  termino_ext == "Con lugar parcial" ').count()

96773

# Formar dataset de sentencias estructurales

In [4]:
from pyspark.sql.types import *
columnas = {
    'se ordena PER' : ArrayType(StringType()),
    'se ordena LOC' : ArrayType(StringType()),
    'se ordena ORG' : ArrayType(StringType()),
    'se ordena MISC' : ArrayType(StringType()),
    'se ordena GPE' : ArrayType(StringType()),
    'se ordena Ent Pub' : ArrayType(StringType())
}
estructurales.separarSeOrdena(columnas, True, True)

columnas = {
    'se condena PER' : ArrayType(StringType()),
    'se condena LOC' : ArrayType(StringType()),
    'se condena ORG' : ArrayType(StringType()),
    'se condena MISC' : ArrayType(StringType()),
    'se condena GPE' : ArrayType(StringType()),
    'se condena Ent Pub' : ArrayType(StringType())
}
estructurales.separarSeCondena(columnas, True, True)

columnas = {
    'extension sentencia' : IntegerType(),
    'extension por lo tanto' : IntegerType()
}
estructurales.extraerExtension(columnas, True)

columnas = {
        'plazosDefinidos' : ArrayType(TimestampType())
    }
    estructurales.plazosDefinidos(columnas, True)

columnas = {
    'FechaSolicitud' : TimestampType(),
}

estructurales.extrarFechaRecibido(columnas, True)

columnas = {
    'num resolucion' : StringType()
}

estructurales.extraerNumeroSentencia(columnas, True)

columnas = {
    'inst internacionales' : ArrayType(StringType())
}

estructurales.extraerInstrumentosInternacionales(columnas, True)

columnas = {
    'derechos Norm' : ArrayType(StringType()),
    'derechos GenXPat' : ArrayType(StringType())
}
estructurales.extraerDerechos(columnas, True)

columnas = {
    'derechos Acotados' : ArrayType(StringType()),
    'derechos General' : ArrayType(StringType()),
    'derechos Fundamental' : ArrayType(StringType()),
    'derechos Humano' : ArrayType(StringType())
}

estructurales.extraerDerechosSinNormalizar(columnas, True)

columnas = {
    'citasFechas' : MapType(StringType(), TimestampType())
}
estructurales.extrarCitaSentenciasFecha(columnas, True)

sentencias = pd.read_csv("listaSentencias.csv", sep=';', encoding='latin-1')
columnas = {
    'id_sentencia' : StringType(),
    'num_doc_oficial' : StringType(),
    'expediente_oficial' : StringType()
}
estructurales.agregarIDSentencia(columnas, sentencias, True)
#Sobreescribir el dataset de filtro de sentencias con las nuevas columnas
estructurales.seleccion.guardarDatos(parquet_file='extracciones2', partition_cols=['anno'], borrar=False)

21/11/24 16:05:33 WARN DAGScheduler: Broadcasting large task binary with size 1151.1 KiB
21/11/24 16:22:02 WARN MemoryStore: Not enough space to cache rdd_58_10 in memory! (computed 263.5 MiB so far)
21/11/24 16:22:02 WARN BlockManager: Persisting block rdd_58_10 to disk instead.
21/11/24 16:54:59 WARN MemoryStore: Not enough space to cache rdd_97_6 in memory! (computed 264.1 MiB so far)
21/11/24 16:54:59 WARN BlockManager: Persisting block rdd_97_6 to disk instead.
21/11/24 21:35:46 WARN DAGScheduler: Broadcasting large task binary with size 1329.3 KiB


In [6]:
estructurales.seleccion.sdf.limit(10).toPandas().columns

Index(['archivo', 'txt', 'cleanText', 'num', 'fechahora_ext', 'redactor_ext',
       'tipoResolucion_ext', 'termino_ext', 'voto_salvado', 'tema',
       'Recurrente_ents', 'Cantón_ents', 'Fecha_ents', 'Constitución_ents',
       'Redactor_ents', 'Ley_ents', 'EntidadPublica_ents', 'tema_prob',
       'termino_lst', 'expediente', 'Fechahora_ents', 'CitaSentencias_ents',
       'Recurrido_ents', 'Reglamento_ents', 'Magistrado_ents', '_id', 'anno',
       'seguimiento', 'se_ordena', 'se_condena', 'plan', 'plazo'],
      dtype='object')

In [16]:
spark.read.parquet('./datasets/estructurales2/extracciones2').filter('termino_ext == "Con lugar" OR  termino_ext == "Con lugar parcial" ').columns

['archivo',
 'txt',
 'cleanText',
 'num',
 'fechahora_ext',
 'redactor_ext',
 'tipoResolucion_ext',
 'termino_ext',
 'voto_salvado',
 'tema',
 'Recurrente_ents',
 'Cantón_ents',
 'Fecha_ents',
 'Constitución_ents',
 'Redactor_ents',
 'Ley_ents',
 'EntidadPublica_ents',
 'tema_prob',
 'termino_lst',
 'expediente',
 'Fechahora_ents',
 'CitaSentencias_ents',
 'Recurrido_ents',
 'Reglamento_ents',
 'Magistrado_ents',
 '_id',
 'seguimiento',
 'se_ordena',
 'se_condena',
 'plan',
 'plazo',
 'se_ordena_PER',
 'se_ordena_LOC',
 'se_ordena_ORG',
 'se_ordena_MISC',
 'se_ordena_GPE',
 'se_ordena_Ent_Pub',
 'se_condena_PER',
 'se_condena_LOC',
 'se_condena_ORG',
 'se_condena_MISC',
 'se_condena_GPE',
 'se_condena_Ent_Pub',
 'extension_sentencia',
 'extension_por_lo_tanto',
 'plazosDefinidos',
 'FechaSolicitud',
 'num_resolucion',
 'inst_internacionales',
 'derechos_Norm',
 'derechos_GenXPat',
 'derechos_Acotados',
 'derechos_General',
 'derechos_Fundamental',
 'derechos_Humano',
 'citasFechas',
 '

# Cargar procesamiento de sentencias estructurales.

In [14]:
terminos = {
    'seguimiento': [r'\bseguimiento\b'],
    'se ordena': [r'\bse ordena\b'],
    'plan': [r'\bplan\b'],
    'plazo': [r'\bplazo\b']
}
seleccion = Seleccion(terminos, spark, parquet_path='../../src/datasets/complete', datasets_path='./datasets/estructurales2/extracciones2')
seleccion.cargarPreprocesados()

estructurales = SentenciasEstructurales(seleccion)
seleccion.sdf.columns

['archivo',
 'txt',
 'cleanText',
 'num',
 'fechahora_ext',
 'redactor_ext',
 'tipoResolucion_ext',
 'termino_ext',
 'voto_salvado',
 'tema',
 'Recurrente_ents',
 'Cantón_ents',
 'Fecha_ents',
 'Constitución_ents',
 'Redactor_ents',
 'Ley_ents',
 'EntidadPublica_ents',
 'tema_prob',
 'termino_lst',
 'expediente',
 'Fechahora_ents',
 'CitaSentencias_ents',
 'Recurrido_ents',
 'Reglamento_ents',
 'Magistrado_ents',
 '_id',
 'anno']

In [ ]:
from pyspark.sql.types import *
columnas = {
    'num resolucion' : StringType()
}

estructurales.extraerNumeroSentencia(columnas, True)

# Zona de trabajo (ignorar)

In [ ]:
s = estructurales.seleccion.sdf
#print(solo_portanto(s.where(s.plazosDefinidos.isNotNull()).limit(15).toPandas().loc[1,'txt']))

In [ ]:
#print(s.where(s.plazosDefinidos.isNotNull()).limit(15).toPandas().loc[1,'se_ordena_PER'])

In [ ]:
#print(s.where(s.plazosDefinidos.isNotNull()).limit(15).toPandas().loc[1,'plazosDefinidos'])

In [ ]:
#print(s.where(s.plazosDefinidos.isNotNull()).limit(15).toPandas().loc[1])